## Setup

In [61]:
import numpy as np
import pandas as pd
import torch
import nltk
import zipfile
import fasttext

from pathlib import Path
from InferSent.models import InferSent
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

nltk.download('punkt')
nltk.download('stopwords')
set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gmvargas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gmvargas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [2]:
# clone or pull InferSent
if not Path("InferSent").is_dir():
    !git clone https://github.com/facebookresearch/InferSent.git
else:
    %cd InferSent
    !git pull
    %cd ..

# download and preprocess SNLI/MultiNLI datasets for fastText
if not Path("InferSent/dataset/fastText").exists():
    %cd Infersent/dataset
    %mkdir fastText
    %cd ..
    %cd ..
    
if not Path("InferSent/dataset/fastText/crawl-300d-2M.vec.zip").is_file():
    !curl -Lo dataset/fastText/crawl-300d-2M.vec.zip https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M.vec.zip

if not Path("InferSent/encoder/infersent2.pkl").is_file():
    !curl -Lo InferSent/encoder/infersent2.pkl https://s3.amazonaws.com/senteval/infersent/infersent2.pkl

C:\Users\gmvargas\Source\repos\MLND\project\InferSent
Already up to date.
C:\Users\gmvargas\Source\repos\MLND\project


In [3]:
if not Path("InferSent/dataset/fastText/crawl-300d-2M.vec/crawl-300d-2M.vec").is_file():
    zipfile.ZipFile("InferSent/dataset/fastText/crawl-300d-2M.vec.zip").extractall(".").close()

In [4]:
# Load pretrained InferSent model

infersent_version = 2
INFERSENT_PATH = "InferSent/encoder/infersent%s.pkl" % infersent_version
infersent_params = {
    'bsize': 64,
    'word_emb_dim': 300,
    'enc_lstm_dim': 2048,
    'pool_type': 'max',
    'dpout_model': 0.0,
    'version': infersent_version
}
infersent = InferSent(infersent_params)
infersent.load_state_dict(torch.load(INFERSENT_PATH))

In [5]:
# setting the word embeddings path to fastText
infersent.set_w2v_path("InferSent/dataset/fastText/crawl-300d-2M.vec/crawl-300d-2M.vec")

In [6]:
# Load embeddings of K most frequent words

infersent.build_vocab_k_words(K=100000)

Vocab size : 100000


## Loading the data

In [7]:
raw_data = pd.read_csv('data/Combined_News_DJIA.csv')

raw_data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [8]:
raw_data_dropped = raw_data.dropna()

## Preprocessing

In [9]:
# stop word removal function

def generate_stop_word_free_dataset(dataframe):
    df = dataframe.copy()
    # iterate over every column
    for column in df.columns:
        # convert column values into an array where each row is an element in the array
        sentences = df[column].tolist()
        # remove stop words
        if isinstance(sentences[0], str): # doesn't seem to be working properly
            stop_wordless_sentences = []
            for sentence in sentences:
                split_sentence = sentence.split()
                new_words = [word for word in split_sentence if word not in stopwords.words('english')]
                # reconstruct the sentence
                new_sentence = " ".join(new_words)
                stop_wordless_sentences.append(new_sentence)
            # recolumnize into dataframe
            df[column] = stop_wordless_sentences
    return(df)

In [10]:
# creating our bearish holdout set for later testing

raw_data_dropped = raw_data_dropped.set_index(['Date']) # sets index to the date column so we can split by date ranges
bearish_raw_data = raw_data_dropped.loc['2008-08-08':'2009-03-31']
bullish_raw_data = raw_data_dropped.loc['2009-04-01':'2016-07-01']

# resets all the indexes just because I feel like it
raw_data_dropped = raw_data_dropped.reset_index()
bearish_raw_data = bearish_raw_data.reset_index()
bullish_raw_data = bullish_raw_data.reset_index()
bullish_raw_data_labels = bullish_raw_data.pop('Label')
bearish_raw_data_labels = bearish_raw_data.pop('Label')

# generate stop wordless datasets

bullish_stop_words_free_data = generate_stop_word_free_dataset(bullish_raw_data)
bearish_stop_words_free_data = generate_stop_word_free_dataset(bearish_raw_data)

In [43]:
def numpy_fillna(data):
    # Get lengths of each row of data
    lens = np.array([len(i) for i in data])

    # Mask of valid places in each row
    mask = np.arange(lens.max()) < lens[:,None]

    # Setup output array and put elements from data into masked positions
    out = np.zeros(mask.shape, dtype=data.dtype)
    out[mask] = np.concatenate(data)
    return out

In [50]:
def convert_sentences_to_vector(dataframe):
    df = dataframe.copy()
    # iterate over every column
    for column in df.columns:
        # convert column values into an array where each row is an element in the array
        sentences = df[column].tolist()
        sentences_vectors = infersent.encode(sentences)
        sentences_vectors_filled = numpy_fillna(sentences_vectors) # fill jagged array, just in case
        for idx, sentence_vector in enumerate(sentences_vectors_filled):
            df.at[idx, column] = np.mean(sentence_vector)
    return df

In [51]:
# preprocessing_pipeline = Pipeline([
#  ('vectorization', FunctionTransformer(convert_sentences_to_vector))   
# ])

In [52]:
data = convert_sentences_to_vector(bullish_raw_data)

In [53]:
data

,Date,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,-0.0602584,-0.011932,0.0113689,0.0108628,0.00514765,0.0110365,0.0172458,0.0212586,0.00124216,0.00195855,...,-0.00679018,-0.00270301,0.0041623,0.00451721,0.0107898,-0.000476476,-0.00516173,0.00584359,0.0131047,0.00504462
1,-0.0602584,0.00926784,0.0226469,0.0180523,0.00354725,0.00505142,0.0169512,0.00870623,0.00483586,0.0177891,...,-0.00467167,0.004051,0.0162227,0.00153858,0.0103432,-0.00340068,0.0067179,0.00566281,0.00232561,-0.00170306
2,-0.0602584,0.00477316,0.0202899,0.00379053,0.0120058,0.0247297,0.00805047,-0.00148465,0.0172585,0.000494899,...,-0.000378695,0.00149113,0.0208021,0.0128213,0.0218717,0.000601104,-0.000915065,0.0124106,0.00316004,-0.0117679
3,-0.0602584,-0.00313358,0.00333734,-0.00359413,-0.000792278,-0.00416229,0.0161665,0.0109843,0.00602853,0.0187776,...,0.00724388,0.0125167,0.0129655,-0.00285159,0.0108403,-0.00543687,-0.00235823,0.0143072,0.0167195,0.010232
4,-0.0602584,-0.00997151,0.0175224,0.00868782,0.0151472,0.00332156,0.00771171,0.00999815,0.0165517,0.0124318,...,0.0264488,0.00787309,-0.0108717,0.0180859,0.00246036,-0.00627531,-0.00717613,0.00190748,-0.00938036,0.0203717
5,-0.0602584,0.0131111,0.0212442,0.00347335,0.00144149,0.00588355,0.0116327,-0.0139333,0.0153012,0.00438989,...,-0.00241375,0.00465097,0.000279919,0.0134379,-0.00913609,-0.0150404,0.00166718,0.00807383,0.0123365,0.00587884
6,-0.0602584,0.000538311,0.00668831,-0.00108646,0.0134473,0.0115851,0.0167857,0.0241819,0.0027264,0.00690218,...,0.0213556,0.00650723,0.00798629,0.0167681,0.0288107,0.00219598,4.77209e-05,0.000159243,0.00755189,0.000420455
7,-0.0602584,0.0112197,-0.0019542,0.00622257,0.0112518,0.0189706,0.0137895,0.00650265,-0.00283514,0.006158,...,0.0047845,0.00918676,0.0062416,0.00400612,0.0103166,-0.0128603,-0.00329642,-0.00269182,-0.00633892,0.0153719
8,-0.0602584,0.000157089,0.0183964,0.0170499,-0.0017138,-0.00726863,0.0101222,0.0122201,0.0128552,0.0213553,...,0.0149939,0.0190476,0.00442083,0.00847461,0.0150603,0.0131898,0.0103962,0.00890485,-0.00474489,0.0023303
9,-0.0602584,-0.000276892,0.00943071,0.00827465,0.00833259,0.0148386,0.0106405,-0.0109925,0.0187327,-0.0011841,...,0.0156131,0.00357666,0.00810247,-0.00551137,0.0120866,0.00783246,0.000768399,-0.0027894,-0.00713074,-0.00332665


In [54]:
# splitting data before preprocessing so as to avoid introducing unseen information from the test set into the training

def split_data(x_data, y_data, test_percent_size):
    return train_test_split(x_data, y_data, test_size=test_percent_size)

x_train, x_test, y_train, y_test = split_data(data, bullish_raw_data_labels, 0.2)

In [56]:
model = SVC(gamma='auto')
model.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [57]:
y_pred = model.predict(x_test)

In [60]:
# accuracy
accuracy = accuracy_score(y_test, y_pred)

# precision
precision = precision_score(y_test, y_pred)

# recall
recall = recall_score(y_test, y_pred)

# f1
f1 = f1_score(y_test, y_pred)

# confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)

Accuracy:  0.5698630136986301
Precision:  0.5698630136986301
Recall:  1.0
F1:  0.7260034904013962
